In [17]:
import sys
sys.path.append('../')
from utils import read_pickle, write_pickle
import numpy as np

In [183]:
sim_code = 'sf'
path = '/Users/diegofiori/Desktop/epfl/master_thesis/results/'

In [184]:
equi_class_no_com = read_pickle(path=path+f'selected_equi_class_{sim_code}_no_comp.pickle')
#equi_class_single = read_pickle(path=path+f'selected_equi_class_{sim_code}_single_field.pickle')
equi_class_norm = read_pickle(path=path+f'selected_equi_class_{sim_code}.pickle')

In [5]:
def union_equi_classes(equi_1, equi_2):
    union_eq_classes = {i: equi_2[key] for i, key in enumerate(equi_2.keys())}
    class_idx = len(union_eq_classes)
    for key_1 in equi_1.keys():
        set_temp_1 = set(equi_1[key_1])
        for key_2 in equi_2.keys(): 
            set_res = set(equi_2[key_2]).intersection(set_temp_1)
            if len(set_res) == 0:
                union_eq_classes[class_idx] = equi_1[key_1]
                class_idx += 1
                break
    return union_eq_classes

def intersect_equi_classes(equi_1, equi_2):
    common_eq_classes = {}
    class_idx = 0
    for key_1 in equi_1.keys():
        set_temp_1 = set(equi_1[key_1])
        for key_2 in equi_2.keys(): 
            set_res = set(equi_2[key_2]).intersection(set_temp_1)
            if len(set_res) > 0:
                common_eq_classes[class_idx] = list(set_res)
                class_idx += 1
    return common_eq_classes

In [185]:
#print(len(equi_class_norm), len(equi_class_no_com), len(equi_class_single))
print(len(equi_class_norm), len(equi_class_no_com))


129 216


In [186]:
#intersection_single = intersect_equi_classes(equi_class_norm, equi_class_single)
intersection_comp = intersect_equi_classes(equi_class_norm, equi_class_no_com)

In [187]:
#print(len(intersection_single), len(intersection_comp))
print(len(intersection_comp))

118


In [188]:
#union_single = union_equi_classes(equi_class_norm, equi_class_single)
union_comp = union_equi_classes(equi_class_norm, equi_class_no_com)

In [189]:
#print(len(union_single), len(union_comp))
print(len(union_comp))

345


# Train Model

single train

In [20]:
sim_code_train = 'eml'
X_train_idx = read_pickle(path=path+f'selected_equi_class_{sim_code_train}.pickle')
idx_temp = []
for key in X_train_idx.keys():
    idx_temp += X_train_idx[key]
X_train_idx = list(set(idx_temp))
X_train = read_pickle(path=path+f'slices_top_features_end_{sim_code_train}.pickle')
X_train = np.concatenate([X_train[i, :, j].reshape(-1, 1) for i, j in X_train_idx], axis=1)

In [21]:
X_train.shape

(7680, 62)

In [24]:
from scipy.io import loadmat
sim_code_k = sim_code
if sim_code.split('_')[-1] == 'comp':
    sim_code_k = sim_code.split('_')[0]
x = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code_k}.mat')
y_train = x['k_quantities']

In [25]:
y_train = np.concatenate([y_train[i, :, :] for i in range(y_train.shape[0])])

In [27]:
y_train.shape

(8000, 2)

In [28]:
y_train = y_train[:len(X_train)]

In [191]:
def get_all_indeces(sim_codes):
    indeces = []
    for sim in sim_codes:
        dict_temp = read_pickle(path=path+f'selected_equi_class_{sim}.pickle')
        for key in dict_temp.keys():
            indeces += dict_temp[key] 
    return list(set(indeces))

In [195]:
def get_data(sim_codes, path, indx=None):
    Xs = []
    ys = []
    if indx is None:
        indx = get_all_indeces(sim_codes)
    for sim_code in sim_codes:
        x_temp = read_pickle(path=path+f'slices_top_features_end_{sim_code}.pickle')
        x_temp = np.concatenate([x_temp[i, :, j].reshape(-1, 1) for i, j in indx], axis=1)
        Xs.append(x_temp)
        
        sim_code_k = sim_code
        if sim_code.split('_')[-1] == 'comp':
            sim_code_k = sim_code.split('_')[0]
        temp = loadmat(f'/Users/diegofiori/Desktop/epfl/master_thesis/k_perp_end_{sim_code_k}.mat')
        y_temp = temp['k_quantities']
        y_temp = np.concatenate([y_temp[i, :, :] for i in range(y_temp.shape[0])])
        y_temp = y_temp[:len(x_temp)]
        ys.append(y_temp)
    return np.concatenate(Xs), np.concatenate(ys)

In [108]:
sim_code_train = 'eml'

In [274]:
train_sims = ['eml', 'hs']
X_train, y_train =  get_data(train_sims, path)

In [275]:
test_sims = ['ds', 'nu2', 'qs']
X_test, y_test = get_data(test_sims, path, 
                          indx=get_all_indeces(train_sims)
                         )

In [276]:
from sklearn.ensemble import RandomForestRegressor

In [277]:
model_1 = RandomForestRegressor(n_estimators=2000, n_jobs=-1)
model_2 = RandomForestRegressor(n_estimators=2000, n_jobs=-1)

In [278]:
model_1.fit(X_train, y_train[:, 0])
model_1.score(X_test, y_test[:, 0])

-4.317081627036832

In [279]:
from sklearn.metrics import mean_absolute_error

In [280]:
def get_prediction_per_sim(model, x_test, y_test, test_sims, n_samples=7680):
    for i, sim in enumerate(test_sims):
        print(f"evaluating simulation {sim}")
        y_temp = y_test[i*n_samples: (i+1)*n_samples]
        x_temp = x_test[i*n_samples: (i+1)*n_samples]
        score = model.score(x_temp, y_temp)
        print(f"score: {score}")
        MAE = mean_absolute_error(y_temp, model.predict(x_temp))
        print(f"MAE: {MAE}\nMAE%: {MAE/y_temp.mean()}")
        if len(y_test.shape) > 1 and y_test.shape[1] > 1:
            double_mae = np.abs(model.predict(x_temp) - y_temp).mean(axis=0)
            print(f'double MAE: {double_mae}')
            print(f'double MAE%: {double_mae/y_temp.mean(axis=0)}')
        

In [281]:
y_test.shape

(23040, 2)

In [282]:
get_prediction_per_sim(model_1, X_test, y_test[:, 0], test_sims)

evaluating simulation ds
score: -7.45671741556218
MAE: 0.010945440802298848
MAE%: 0.39033259994055863
evaluating simulation nu2
score: -2.4669772417319584
MAE: 0.004853617906164219
MAE%: 0.18422274425119115
evaluating simulation qs
score: -1.7881699140395464
MAE: 0.004374697804151178
MAE%: 0.1561302389633777


In [ ]:
model_2.fit(X_train, y_train[:, 1])

In [ ]:
get_prediction_per_sim(model_2, X_test, y_test[:, 1], test_sims)

In [ ]:
model = RandomForestRegressor(n_estimators=2000, n_jobs=-1)

In [ ]:
model.fit(X_train, y_train)
model.score(X_test, y_test)

In [ ]:
get_prediction_per_sim(model, X_test, y_test, test_sims)